<a href="https://colab.research.google.com/github/grace3999/PPs/blob/master/Kaggle_GOSSIS_comb_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
#install packages
!pip install category-encoders

In [0]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import scipy as sp
import missingno as msno
import pickle

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster', rc={'font.size':35,
                              'axes.titlesize':50,
                              'axes.labelsize':35})

#machine learning
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.preprocessing import StandardScaler, Normalizer, LabelEncoder, PolynomialFeatures
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, train_test_split, cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.svm import SVC 
import xgboost as xgb
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, roc_auc_score, classification_report
from sklearn.feature_selection import RFECV

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [0]:
#data paths
train_data_path = '/content/gdrive/My Drive/Colab_projects/GOSSIS_kaggle/training_v2.csv'
unlabeled_data_path = '/content/gdrive/My Drive/Colab_projects/GOSSIS_kaggle/unlabeled.csv'

In [5]:
data = pd.read_csv(train_data_path)
data_train = pd.DataFrame(data = data)

print('Original data shape:\n', data_train.shape, '\n')
print('Group value counts:\n', data_train['hospital_death'].value_counts(), '\n')

#drop meta_data
data_meta = ['patient_id', 'hospital_id', 'icu_id']
data_train = data_train.drop(columns = data_meta)

#add column naming data set
data_train['data_set'] = 'train'

#save hospital death data in separate df
data_train_death = data_train[['encounter_id', 'hospital_death']]
print('Death info data shape:\n', data_train_death.shape, '\n')

Original data shape:
 (91713, 186) 

Group value counts:
 0    83798
1     7915
Name: hospital_death, dtype: int64 

Death info data shape:
 (91713, 2) 



In [6]:
data = pd.read_csv(unlabeled_data_path)
data_unlabled = pd.DataFrame(data = data)

print('Original data shape:\n', data_unlabled.shape, '\n')
print('Group value counts:\n', data_unlabled['hospital_death'].value_counts(), '\n')

#drop meta_data
data_meta = ['patient_id', 'hospital_id', 'icu_id']
data_unlabled = data_unlabled.drop(columns = data_meta)

#add column naming data set
data_unlabled['data_set'] = 'unlabled'

data_unlabled.head()

Original data shape:
 (39308, 186) 

Group value counts:
 Series([], Name: hospital_death, dtype: int64) 



,encounter_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,data_set
0,2,NaN,56.0,21.102439,0,Caucasian,M,180.3,Emergency Department,Accident & Emergency,admit,Neuro ICU,0.104167,0,68.6,3.7,301.0,405.01,0,0.0,NaN,19.0,0.67,NaN,2.0,5.0,0.0,1.0,276.0,105.0,42.0,0.0,57.0,NaN,NaN,NaN,NaN,5.0,134.0,36.6,1121.1264,0.0,4.7,NaN,NaN,74.0,51.0,74.0,51.0,100.0,78.0,NaN,NaN,86.0,60.0,86.0,60.0,20.0,8.0,100.0,96.0,NaN,NaN,121.0,79.0,121.0,79.0,37.1,36.6,NaN,NaN,55.0,51.0,55.0,51.0,88.0,84.0,NaN,NaN,68.0,60.0,68.0,60.0,18.0,8.0,100.0,98.0,NaN,NaN,91.0,79.0,91.0,79.0,36.6,36.6,3.7,3.7,NaN,NaN,19.0,19.0,9.3,9.3,0.67,0.67,276.0,167.0,24.0,24.0,13.8,13.8,42.0,42.0,NaN,NaN,NaN,NaN,223.0,223.0,4.8,4.8,135.0,134.0,4.7,4.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.13,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic,unlabled
1,5,NaN,NaN,19.500595,1,Caucasian,F,145.0,Operating Room,Operating Room / Recovery,admit,Med-Surg ICU,0.

In [9]:
param_cat = ['ethnicity', 'gender', 'icu_admit_source', 'hospital_admit_source', 'icu_stay_type', 'icu_type', 
       'apache_3j_bodysystem', 'apache_2_bodysystem']

#encode categorical variables using targetencoder - will need to retrain on fianl feature set
data_cat_train = data_train.copy()

Y_train = data_cat_train['hospital_death']
X_train = data_cat_train

# use target encoding to encode two categorical features
enc = ce.TargetEncoder(cols=param_cat).fit(X_train, Y_train)
cat_enc_data = enc.transform(X_train)
print(cat_enc_data.shape)
#use target encoding model to encode categorical features of unlabled data
cat_data_unlabled = enc.transform(data_unlabled)
print(cat_data_unlabled.shape)

(91713, 184)
(39308, 184)


In [10]:
#combine data sets for nan imputation and normalization
comb_data_cat = pd.concat([cat_enc_data, cat_data_unlabled], axis=0)
print(comb_data_cat.shape)
comb_data_cat.head()

(131021, 184)


,encounter_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,data_set
0,66154,0.0,68.0,22.73,0,0.087262,0.084417,180.3,0.138796,0.134136,0.085168,0.060205,0.541667,0,73.9,2.3,113.0,502.01,0,0.0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0.0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,NaN,0.0,14.1,46.0,32.0,68.0,37.0,68.0,37.0,119.0,72.0,66.0,40.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,122.0,64.0,131.0,73.0,131.0,73.0,39.9,37.2,NaN,NaN,68.0,63.0,68.0,63.0,119.0,108.0,NaN,NaN,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,NaN,NaN,131.0,115.0,131.0,115.0,39.5,37.5,2.3,2.3,0.4,0.4,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,NaN,NaN,1.3,1.0,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.157922,0.108383,train
1,114252,0.0,77.0,27.42,0,0.087262,0.088373,160.0,0.138796,0.134136,0.085168,0.087495,0.927778,0,70.2,NaN,108.0,203.01

In [98]:
features_orig = ['data_set', 'hospital_death', 'ethnicity', 'gender', 'icu_admit_source',
       'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'age', 'bmi',
       'apache_4a_icu_death_prob', 'fio2_apache', 'gcs_eyes_apache',
       'gcs_motor_apache', 'gcs_verbal_apache', 'urineoutput_apache',
       'ventilated_apache', 'elective_surgery', 'intubated_apache',
       'd1_diasbp_min',
       'd1_heartrate_max', 
       'd1_mbp_min',
       'd1_resprate_max',  
       'd1_spo2_min',
       'd1_sysbp_min',
       'd1_temp_min',
       'd1_albumin_max', 'd1_albumin_min', 'd1_bilirubin_max',
       'd1_bilirubin_min', 'd1_bun_max', 'd1_bun_min', 
       'd1_calcium_min', 
       'd1_creatinine_max', 'd1_creatinine_min',
       'd1_hco3_max', 'd1_hco3_min',
       'd1_inr_max', 'd1_inr_min', 'd1_lactate_max', 'd1_lactate_min', 
       'd1_potassium_max', 
       'd1_wbc_max', 'd1_wbc_min',
       'd1_pao2fio2ratio_min']

features_models = ['apache_4a_hospital_death_prob', 'age', 'd1_lactate_max',
       'creatinine_apache', 'd1_heartrate_min', 'bmi', 'd1_platelets_max',
       'd1_spo2_min', 'wbc_apache', 'pre_icu_los_days', 'glucose_apache',
       'urineoutput_apache', 'd1_inr_max', 'bun_apache', 'd1_sysbp_max',
       'temp_apache', 'd1_hco3_max', 'd1_temp_max', 'hematocrit_apache',
       'd1_diasbp_min', 'd1_resprate_max', 'd1_glucose_min',
       'd1_arterial_ph_max', 'map_apache', 'd1_arterial_po2_max',
       'd1_resprate_min', 'h1_temp_max', 'bilirubin_apache',
       'd1_pao2fio2ratio_min', 'h1_diasbp_min', 'sodium_apache',
       'apache_2_diagnosis', 'd1_potassium_min', 'resprate_apache',
       'h1_diasbp_max', 'hospital_death']

features_short = ['encounter_id', 'data_set', 'hospital_death', 'fio2_apache', 'ventilated_apache', 'apache_4a_icu_death_prob', 'd1_lactate_max', 'd1_lactate_min', 'gcs_motor_apache',
                  'd1_sysbp_min', 'd1_spo2_min']

#save meta data
comb_data_cat_meta = comb_data_cat[['encounter_id', 'data_set', 'hospital_death']]

comb_data_cat_final = comb_data_cat[features_short].drop(columns = ['encounter_id', 'hospital_death', 'data_set'], axis=1)

comb_data_cat_final.head()

,fio2_apache,ventilated_apache,apache_4a_icu_death_prob,d1_lactate_max,d1_lactate_min,gcs_motor_apache,d1_sysbp_min,d1_spo2_min
0,NaN,0.0,0.05,1.3,1.0,6.0,73.0,74.0
1,1.0,1.0,0.29,3.5,3.5,3.0,67.0,70.0
2,NaN,0.0,0.00,NaN,NaN,6.0,105.0,91.0
3,0.6,1.0,0.03,NaN,NaN,6.0,84.0,95.0
4,NaN,0.0,NaN,NaN,NaN,NaN,120.0,96.0


In [85]:
#impute missing data for features

imp = IterativeImputer(max_iter=99, random_state=39, verbose=2, skip_complete=True).fit(comb_data_cat_final)
comb_data_cat_final_impute = imp.transform(comb_data_cat_final)

[IterativeImputer] Completing matrix with shape (131021, 8)
[IterativeImputer] Ending imputation round 1/99, elapsed time 0.37
[IterativeImputer] Change: 25.721066285839754, scaled tolerance: 0.16 
[IterativeImputer] Ending imputation round 2/99, elapsed time 0.76
[IterativeImputer] Change: 36.107924928087016, scaled tolerance: 0.16 
[IterativeImputer] Ending imputation round 3/99, elapsed time 1.15
[IterativeImputer] Change: 25.486119837855444, scaled tolerance: 0.16 
[IterativeImputer] Ending imputation round 4/99, elapsed time 1.52
[IterativeImputer] Change: 14.782956229849615, scaled tolerance: 0.16 
[IterativeImputer] Ending imputation round 5/99, elapsed time 1.89
[IterativeImputer] Change: 7.845837199808239, scaled tolerance: 0.16 
[IterativeImputer] Ending imputation round 6/99, elapsed time 2.25
[IterativeImputer] Change: 3.9055208403467403, scaled tolerance: 0.16 
[IterativeImputer] Ending imputation round 7/99, elapsed time 2.62
[IterativeImputer] Change: 1.75295583041337, s

In [101]:
#scale data algo
scaler = StandardScaler()

comb_data_cat_final_impute_scaled = scaler.fit_transform(comb_data_cat_final_impute)

#create final data set
comb_data_cat_final_impute_scaled = pd.DataFrame(data=comb_data_cat_final_impute_scaled, columns=comb_data_cat_final.columns.values)

comb_data_cat_final_impute_scaled['data_set'] = comb_data_cat_meta['data_set'].values
comb_data_cat_final_impute_scaled['hospital_death'] = comb_data_cat_meta['hospital_death'].values
comb_data_cat_final_impute_scaled['encounter_id'] = comb_data_cat_meta['encounter_id'].values

comb_data_cat_final_impute_scaled.head()

,fio2_apache,ventilated_apache,apache_4a_icu_death_prob,d1_lactate_max,d1_lactate_min,gcs_motor_apache,d1_sysbp_min,d1_spo2_min,data_set,hospital_death,encounter_id
0,-0.462241,-0.704830,0.035057,-0.550692,-0.581911,0.405136,-1.146253,-1.614082,train,0.0,66154
1,3.125092,1.427654,1.152969,0.622705,1.433293,-1.958287,-1.436564,-2.008442,train,0.0,114252
2,-0.443114,-0.704830,-0.197841,-0.264072,-0.221835,0.405136,0.402076,0.061949,train,0.0,119783
3,0.706278,1.427654,-0.058102,0.074438,0.001917,0.405136,-0.614015,0.456309,train,0.0,79267
4,-0.634431,-0.704830,-0.353641,-0.593126,-0.550357,0.381418,1.127854,0.554900,train,0.0,92056


In [0]:
#separate data sets 
train_data_final = comb_data_cat_final_impute_scaled[comb_data_cat_final_impute_scaled['data_set'] == 'train'].drop(columns='data_set', axis=1)
unlabeled_data_final = comb_data_cat_final_impute_scaled[comb_data_cat_final_impute_scaled['data_set'] == 'unlabled'].drop(columns='data_set', axis=1)

# shallow learning


In [0]:

#split data
train, test = train_test_split(train_data_final.copy(), test_size = .9, random_state=1, stratify = train_data_final['hospital_death'])

Y_train = train['hospital_death']
Y_test = test['hospital_death']


X_train = train.drop('hospital_death',axis = 1)
X_test = test.drop('hospital_death',axis = 1)


In [0]:
#k fold algo
strat_k_fold = StratifiedKFold(n_splits=10)

#classifier algos
dm_cv = DummyClassifier(strategy='stratified', random_state=39)
lr_cv = LogisticRegression(random_state=39, class_weight='balanced')
rf_cv = RandomForestClassifier(random_state=39, class_weight='balanced')
knn_cv = KNeighborsClassifier()
ab_cv = AdaBoostClassifier(random_state=39)
gb_cv = GradientBoostingClassifier(random_state=39)
svm_cv = SVC(kernel='rbf', probability=False, class_weight='balanced') 
lgb_cv = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', n_estimators = 10000, class_weight = 'balanced')

#dic with classifier and feature importance attribute name
models_dic = {'dm_cv': (dm_cv, 'none'), 
              'lr_cv': (lr_cv, 'coef'), 
              'lgb_cv': (lgb_cv, 'none'), 
              'ab_cv': (ab_cv, 'feature_importance')}

#'gb_cv': (gb_cv, 'feature_importance'),
              #'knn_cv': (knn_cv, 'none'),
              #'svm_cv':(svm_cv, 'coef')
              #'rf_cv': (rf_cv, 'feature_importance'),

In [0]:
def feature_importance(X, y, model_instance, feature_names, fi_name):
    #takes in features (X) and classess (y), model, column names for features in X, and name of attribute for feature importance
    #returns dictionary of feature names and coef/feature importance values
    
    feature_importance_dic = {}
    
    model_instance.fit(X, y)
    
    if fi_name == 'coef':
        coef = model_instance.coef_[0]
        feature_importance_dic = dict(zip(feature_names, coef))
    if fi_name == 'feature_importance':
        coef = model_instance.feature_importances_
        feature_importance_dic = dict(zip(feature_names, coef))
    if fi_name == 'none':
        coef = np.zeros(len(feature_names))
        feature_importance_dic = dict(zip(feature_names, coef))
    
    return feature_importance_dic

In [0]:
def classification_pipeline(X, y, cv_instance, model_instance, feature_names, fi_name):
    
    #generate cross-val sets
    cv = list(cv_instance.split(X, y))
    
    #predict class and predict probability 
    y_pred = cross_val_predict(model_instance, X, y, cv=cv, method='predict')
    #y_pred_prob = cross_val_predict(model_instance, X, y, cv=cv, method='predict_proba')
    
    #generate confusion matrix
    conf_mat = confusion_matrix(y, y_pred)
    print('Confusion matrix:', conf_mat)
    
    #generate ROC_AUC
    #ROC_AUC = metrics.roc_auc_score(y, y_pred_prob[:,1])
    #print("ROC_AUC: ", ROC_AUC)
    
    # generate additional metrics
    recall = metrics.recall_score(y,y_pred)
    precision = metrics.precision_score(y,y_pred)
    accuracy = metrics.accuracy_score(y,y_pred)
    F1 = metrics.f1_score(y,y_pred)
    print("Sensitivity/Recall (TPR): ",recall)
    print("Precision (PPV): ", precision)
    print("Accuracy: ", accuracy)
    print("F1:", F1)
    
    #determine feature importance
    #feature_dic = feature_importance(data_scaled, y, model_instance, feature_names, fi_name)
    
    #create dic
    data_dic = {}
    data_dic['y_pred'] = y_pred
    #data_dic['y_pred_prob'] = y_pred_prob
    data_dic['conf_mat'] = conf_mat
    #data_dic['ROC_AUC'] = ROC_AUC
    data_dic['recall'] = recall
    data_dic['precision'] = precision
    data_dic['accuracy'] = accuracy
    data_dic['F1'] = F1
    
    #data_dic = {**data_dic, **feature_dic}
    
    return data_dic

In [95]:
feature_set = 'full'
feature_names = X_train.columns.values

data_full_features = {}

models_dic = {'dm_cv': (dm_cv, 'none'), 
              'lr_cv': (lr_cv, 'coef'),
              'lgb_cv': (lgb_cv, 'none'), 
              'ab_cv': (ab_cv, 'feature_importance')}

for name, model in models_dic.items():
    print(f'{name} model with {feature_set} features:')
    data_full_features[name + '_' + feature_set] = classification_pipeline(X_train, Y_train, strat_k_fold, model[0], feature_names, model[1])
    print('\n')

dm_cv model with full features:
Confusion matrix: [[7559  821]
 [ 713   78]]
Sensitivity/Recall (TPR):  0.0986093552465234
Precision (PPV):  0.0867630700778643
Accuracy:  0.8327336168356777
F1: 0.09230769230769231


lr_cv model with full features:
Confusion matrix: [[6674 1706]
 [ 248  543]]
Sensitivity/Recall (TPR):  0.6864728192161821
Precision (PPV):  0.2414406402845709
Accuracy:  0.7869370842874278
F1: 0.35723684210526313


lgb_cv model with full features:
Confusion matrix: [[8003  377]
 [ 526  265]]
Sensitivity/Recall (TPR):  0.3350189633375474
Precision (PPV):  0.4127725856697819
Accuracy:  0.9015374550212627
F1: 0.3698534542916957


ab_cv model with full features:
Confusion matrix: [[8276  104]
 [ 591  200]]
Sensitivity/Recall (TPR):  0.2528445006321112
Precision (PPV):  0.6578947368421053
Accuracy:  0.9242176425689674
F1: 0.365296803652968




In [0]:
#put dics in pandas df 
final_dic = {**data_full_features}
data_pandas = pd.DataFrame.from_dict(data = final_dic, orient='index')
data_pandas.sort_values('precision', ascending=False).head()

In [97]:
#run on test data with best optimized model
ab_cv.fit(X_train, Y_train)

print('GB test AUC: {}'.format(ab_cv.score(X_test, Y_test)))
test_pred_gb = ab_cv.predict(X_test)
test_pred_prob_gb = ab_cv.predict_proba(X_test)
print(classification_report(Y_test, test_pred_gb))
print(confusion_matrix(Y_test, test_pred_gb))

GB test AUC: 0.9232027331540307
              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96     75418
         1.0       0.65      0.24      0.35      7124

    accuracy                           0.92     82542
   macro avg       0.79      0.62      0.66     82542
weighted avg       0.91      0.92      0.91     82542

[[74468   950]
 [ 5389  1735]]


# Save final models

In [0]:
#pickel model to save for later use with test set
save_path = '/content/gdrive/My Drive/Colab_projects/GOSSIS_kaggle/'

#classification model
pkl_filename = str(save_path + "classification_model_comb.pkl")  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(ab_cv, file)

# Use models on test set


In [105]:
probstest = ab_cv.predict_proba(unlabeled_data_final.copy().drop(columns=['hospital_death', 'encounter_id'], axis=1))
probstest = probstest[:, 1]
unlabeled_data_final["hospital_death"] = probstest
unlabeled_data_final[["encounter_id","hospital_death"]].to_csv(str(save_path + "submissioncomb.csv"),index=False)
unlabeled_data_final[["encounter_id","hospital_death"]].head()

,encounter_id,hospital_death
91713,2,0.487370
91714,5,0.484588
91715,7,0.480679
91716,8,0.493467
91717,10,0.499438


In [0]:
save_path = '/content/gdrive/My Drive/Colab_projects/GOSSIS_kaggle/'
data_test[["encounter_id","hospital_death"]].to_csv(str(save_path + "submissioncomb.csv"),index=False)

In [0]:
#scale data algo
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

eclf2 = VotingClassifier(estimators=[
        ('lr', lr_cv), ('rf', rf_cv), ('gb', gb_cv), ('ab', ab_cv)], 
                         voting='soft', weights=[5,1,1,1])
eclf2 = eclf2.fit(X_train, Y_train)
print('Train AUC with soft voting: ', eclf2.score(X_train, Y_train))
print('Test AUC with soft voting: ', eclf2.score(X_test, Y_test))
test_pred_hard = eclf2.predict(X_test)
test_pred_prob_hard = eclf2.predict_proba(X_test)
print(classification_report(Y_test, test_pred_hard))
print(confusion_matrix(Y_test, test_pred_hard))

Train AUC with soft voting:  0.8819103696434413
Test AUC with soft voting:  0.8688473520249221
              precision    recall  f1-score   support

           0       0.97      0.89      0.93     58659
           1       0.36      0.67      0.47      5541

    accuracy                           0.87     64200
   macro avg       0.66      0.78      0.70     64200
weighted avg       0.91      0.87      0.89     64200

[[52043  6616]
 [ 1804  3737]]
